# NYC MTA Turnstile Dataset Cleanup and Merging Data from Different Sources

In [1]:
from IPython.display import display
import warnings
import calendar
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# Process and dump a single file
def delta_processor(filename):
    df = pd.read_csv(filename)
    
    #Cleanup column names - remove spaces
    df.rename(columns=lambda x: x.strip(), inplace=True)

    # Cleaning datetime
    print("Cleaning up date time")
    df.loc[:,'DATETIME'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'])
    df['DAY'] = df['DATETIME']
    df['DAY'] = df['DAY'].apply(lambda x : calendar.day_name[x.weekday()])
    
    # diff entry and exit and update dataframe
    print('Calculate running delta for entry/exit time')
    entries_diff = df['ENTRIES'].diff()
    entries_diff = entries_diff.clip_lower(0)
    df['ENTRIES_DELTA'] = entries_diff
    exits_diff = df['EXITS'].diff()
    exits_diff = exits_diff.clip_lower(0)
    df['EXITS_DELTA'] = exits_diff
    df.fillna(0, inplace=True)
    
    # First entry/exit delta for each day in each station and SCP will be incorrect. Drop these results
    # Next, there are chances for spurious deltas if data is not correct, ignore such outliers
    print('Removing spurious records')
    df = df.groupby(['STATION', 'SCP', 'DATE'], group_keys=False).apply(lambda group: group.iloc[1:])
    df = df[df['ENTRIES_DELTA'] < 10000]
    df = df[df['EXITS_DELTA'] < 10000]
    
    #TODO: Write to CSV
    # 'abc.csv' -> 'abc_delta_processed.csv'
    return df 
    

In [3]:
turnstile_df = delta_processor('MTA_all_data.csv')

Cleaning up date time
Calculate running delta for entry/exit time
Removing spurious records


In [4]:
turnstile_df[[ 'UNIT','DIVISION', 'STATION', 'DATE', 'TIME','DATETIME', 'DAY', 'ENTRIES_DELTA', 'EXITS_DELTA']].head()

,UNIT,DIVISION,STATION,DATE,TIME,DATETIME,DAY,ENTRIES_DELTA,EXITS_DELTA
1411287,R248,BMT,1 AV,07/28/2018,04:00:00,2018-07-28 04:00:00,Saturday,112.0,239.0
1411288,R248,BMT,1 AV,07/28/2018,08:00:00,2018-07-28 08:00:00,Saturday,77.0,284.0
1411289,R248,BMT,1 AV,07/28/2018,12:00:00,2018-07-28 12:00:00,Saturday,482.0,657.0
1411290,R248,BMT,1 AV,07/28/2018,16:00:00,2018-07-28 16:00:00,Saturday,661.0,780.0
1411291,R248,BMT,1 AV,07/28/2018,20:00:00,2018-07-28 20:00:00,Saturday,511.0,763.0


In [5]:
#  out to csv after preliminary cleaning and diff generation
## turnstile_df.to_csv('MTA_all_data_processed2.csv')

In [6]:
# read in zipcode data
zips = pd.read_csv("station_zips.csv")

In [7]:
# clean up of station names

def fix_names(name):
    temp_name = name.upper()
    replacements = [["1TH", "1"],["1ST", "1"], ["2ND", "2"],["3RD", "3"],["4TH", "4"],["5TH", "5"], 
                    ["6TH", "6"],["7TH", "7"],["8TH", "8"],["9TH", "9"],["0TH", "0"],["COLUMBIA UNIVERSITY", "COLUMBIA"],
                    ["AVENUE", "AV"], ["SQUARE", "SQ"], ["CENTER","CTR"], ["138 ST-3 AVE","3 AV 138 ST"],
                    ["149 ST-3 AV", "3 AV-149 ST"], ["137 ST-CITY COLLEGE", "137 ST CITY COL"], 
                    ["PROSPECT PARK-15 ST","15 ST-PROSPECT"], ['YANKEE STADIUM-161 ST', "161/YANKEE STAD"],
                    ['RAWSON ST-33 ST', "33 ST-RAWSON ST"], ['WHITEHALL ST', "WHITEHALL S-FRY"], 
                    ['WESTCHESTER SQ-EAST TREMONT AV',"WESTCHESTER SQ" ], ['WAKEFIELD-241 ST', "WAKEFIELD/241"],
                    ['WEST 8 ST', "W 8 ST-AQUARIUM"], ['WEST 4 ST', 'W 4 ST-WASH SQ'], 
                    ['VERNON BLVD-JACKSON AV',"VERNON-JACKSON" ],["5 AV-53 ST","5 AV/53 ST"],["5 AV","5 AVE"],
                    ["59 ST-COLUMBUS CIRCLE","59 ST COLUMBUS"],["WOODSIDE AV-61 ST","61 ST WOODSIDE"],
                    ["63 DRIVE-REGO PARK","63 DR-REGO PARK"],["66 ST-LINCOLN CTR","66 ST-LINCOLN"],
                    ["68 ST-HUNTER COLLEGE","68ST-HUNTER CO"],["AQUEDUCT-NORTH CONDUIT AV","AQUEDUCT N.COND"],
                    ["AQUEDUCT RACETRACK","AQUEDUCT RACETR"],["ASTOR PLACE","ASTOR PL"],
                    ["ASTORIA BLVD-HOYT AV","ASTORIA BLVD"],["DITMARS BLVD","ASTORIA DITMARS"],
                    ["ATLANTIC AV-BARCLAYS CTR","ATL AV-BARCLAY"],["BROADWAY-LAFAYETTE ST","B'WAY-LAFAYETTE"],
                    ["BAY PARKWAY","BAY PKWY"],["BEDFORD PARK BLVD","BEDFORD PK BLVD"],
                    ["BEDFORD-NOSTRAND AVS","BEDFORD-NOSTRAN"],["BEVERLY RD","BEVERLEY ROAD"],
                    ["BOTANIC GARDENS","BOTANIC GARDEN"],["BRIARWOOD-VAN WYCK BLVD","BRIARWOOD"],
                    ["BROADWAY JUNCTION-EAST NEW YORK","BROADWAY JCT"],["BROOKLYN BRIDGE-CITY HALL","BROOKLYN BRIDGE"],
                    ["CANARSIE - ROCKAWAY PARKWAY","CANARSIE-ROCKAW"],["CATHEDRAL PARKWAY-110 ST","CATHEDRAL PKWY"],
                    ["CLINTON-WASHINGTON AVS","CLINTON-WASH AV"],["DELANCEY ST","DELANCEY/ESSEX"],
                    ["EAST 143 ST-ST MARY'S ST","E 143/ST MARY'S"],["EAST 149 ST","E 149 ST"],
                    ["EAST 180 ST","E 180 ST"],["EASTCHESTER-DYRE AV","EASTCHSTER/DYRE"],
                    ["EASTERN PARKWAY-BROOKLYN MUSEUM","EASTN PKWY-MUSM"],["HARLEM-148 ST","HARLEM 148 ST"],
                    ["FAR ROCKAWAY-MOTT AV","FAR ROCKAWAY"],["FLATBUSH AV-BROOKLYN COLLEGE","FLATBUSH AV-B.C"],
                    ["FLUSHING-MAIN ST","FLUSHING-MAIN"],["FOREST AV","FOREST AVE"],
                    ["FOREST HILLS-71 AV","FOREST HILLS 71"],["FORT HAMILTON PARKWAY","FT HAMILTON PKY"],
                    ["GRAND ARMY PLAZA","GRAND ARMY PLAZ"],["GRAND AV-NEWTOWN","GRAND-NEWTOWN"],
                    ["HOWARD BEACH","HOWARD BCH JFK"],["HOYT & SCHERMERHORN","HOYT-SCHER"],
                    ["HUNTERS POINT","HUNTERS PT AV"],["INWOOD - 207 ST","INWOOD-207 ST"],
                    ["JAMAICA-179 ST","JAMAICA 179 ST"],["JACKSON HEIGHTS-ROOSEVELT AVE","JKSN HT-ROOSVLT"],
                    ["JAMAICA-VAN WYCK","JAMAICA VAN WK"],["KEW GARDENS-UNION TURNPIKE","KEW GARDENS"],
                    ["KINGS HIGHWAY","KINGS HWY"],["KNICKERBOCKER AV","KNICKERBOCKER"],["KOSCIUSKO ST","KOSCIUSZKO ST"],
                    ["NORWOOD-205 ST","NORWOOD 205 ST"],["OCEAN PARKWAY","OCEAN PKWY"],["PELHAM PARKWAY","PELHAM PKWY"],
                    ["QUEENSBORO PLAZA","QUEENSBORO PLZ"],["PARKCHESTER-EAST 177 ST","PARKCHESTER"],
                    ["LEXINGTON AV-53 ST","LEXINGTON AV/53"],["LEXINGTON AV","LEXINGTON AV/63"],
                    ["MARBLE HILL-225 ST","MARBLE HILL-225"],["METS - WILLETS POINT","METS-WILLETS PT"],
                    ["MORRISON AV-SOUNDVIEW AV","MORISN AV/SNDVW"],["MOSHOLU PARKWAY","MOSHOLU PKWY"],
                    ["MYRTLE-WILLOUGHBY AVS","MYRTLE-WILLOUGH"],["MYRTLE AV","MYRTLE-WYCKOFF"],
                    ["NEPTUNE AV-VAN SICLEN","NEPTUNE AV"],["238 ST-NEREID AV","NEREID AV"],
                    ["ROCKAWAY PARK-BEACH 116","ROCKAWAY PARK B"],["ROOSEVELT ISLAND","ROOSEVELT ISLND"],
                    ["SENECA AV","SENECA AVE"],["SUTPHIN BLVD-ARCHER AV - JFK","SUTPHIN-ARCHER"],
                    ["SUTTER AV","SUTTER AV-RUTLD"],["VAN CORTLANDT PARK-242 ST","V.CORTLANDT PK"],
                    ["VAN SICLEN AV","VAN SICLEN AVE"], ["AV H",'AVENUE H'], ["AV I",'AVENUE I'], ["AV J",'AVENUE J'],
                    ["AV M",'AVENUE M'], ["AV N",'AVENUE N'], ["AV P",'AVENUE P'], ["AV U",'AVENUE U'], 
                    ["AV X",'AVENUE X']]
    
       
    for x in replacements:
        temp_name = temp_name.replace(x[0], x[1])
    return temp_name

In [8]:
zips["STATION"] = zips["STATION"].apply(fix_names)

In [9]:
# merge zip code data with processed turnstile data
zipped_station = pd.merge(turnstile_df[[ 'UNIT','DIVISION', 'STATION', 'DATE', 'TIME','DATETIME', 'DAY', 
                                        'ENTRIES_DELTA', 'EXITS_DELTA']], zips[["DIVISION","STATION","zip"]],
                          on=["DIVISION","STATION"], how="left")


In [10]:
# read in file to map zip codes to csv
manual_zip_df = pd.read_csv("name_zip_mapping.csv")

In [11]:
# fixing/cleaning remaining station names
def fix_manual_names(cols):
    station_name= cols[0]
    zipcode=cols[1]
    if np.isnan(zipcode):
        values = manual_zip_df[manual_zip_df["name"]==station_name]["zip"].values
        if len(values)>0:
            return values[0]
        else:
            return np.nan
    else:
        return zipcode
        

In [12]:
# applying the cleaning function to zip codes and mapping stations to zip codes
zipped_station["zip"] = zipped_station[["STATION", "zip"]].apply(fix_manual_names,  axis=1)

In [13]:
read_dictionary = np.load('zipcode_to_income_dict.npy').item()

In [14]:
# read in income data
def add_income(zip_code):
    if zip in read_dictionary.keys():
        return read_dictionary[zip_code]


In [15]:
zipped_station["income"] = zipped_station["zip"].apply(add_income)

In [16]:
zipped_station.sample(30)

,UNIT,DIVISION,STATION,DATE,TIME,DATETIME,DAY,ENTRIES_DELTA,EXITS_DELTA,zip,income
444728,R084,IND,59 ST COLUMBUS,09/08/2018,12:00:00,2018-09-08 12:00:00,Saturday,580.0,205.0,10023.0,None
858614,R224,IRT,CLARK ST,09/18/2018,04:00:00,2018-09-18 04:00:00,Tuesday,15.0,8.0,11201.0,None
391103,R079,BMT,5 AV/59 ST,08/06/2018,10:00:00,2018-08-06 10:00:00,Monday,32.0,108.0,10023.0,None
1413448,R408,IRT,SIMPSON ST,08/31/2018,17:00:00,2018-08-31 17:00:00,Friday,400.0,64.0,10459.0,None
624270,R263,BMT,AVENUE H,09/06/2018,20:00:00,2018-09-06 20:00:00,Thursday,424.0,66.0,11230.0,None
524055,R085,BMT,8 ST-NYU,08/14/2018,16:00:00,2018-08-14 16:00:00,Tuesday,258.0,91.0,10003.0,None
342594,R011,IND,42 ST-PORT AUTH,08/25/2018,16:00:00,2018-08-25 16:00:00,Saturday,19.0,34.0,10036.0,None
293486,R072,IRT,34 ST-HUDSON YD,09/03/2018,04:00:00,2018-09-03 04:00:00,Monday,0.0,0.0,10001.0,None
975391,R119,IRT,FORDHAM RD,08/17/2018,16:00:00,2018-08-17 16:00:00,Friday,0.0,0.0,10468.0,None
468398,R177,IRT,68ST-HUNTER CO,08/06/2018,21:00:00,2018-08-06 21:00:00,Monday,226.0,50.0,10065.0,None
